# Importing Libraries

In [218]:
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, f1_score
import time
import pickle 
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score , precision_score, recall_score, f1_score

warnings.simplefilter(action='ignore', category=FutureWarning)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Reading Files

In [248]:
crude_oil_prices = pd.read_csv('../InputData/crude_oil_prices.csv')
federal_rates = pd.read_csv('../InputData/effective_federal_funds_rate.csv')
corridor_rates = pd.read_csv('../InputData/egyptian_corridor_interest_rates.csv')
housing_index = pd.read_csv('../InputData/housing_index.csv')
inflation_mom = pd.read_csv('../InputData/inflation_month_on_month.csv')
inflation_yoy = pd.read_csv('../InputData/inflation_year_on_year.csv')
news_data = pd.read_csv('../InputData/news.csv')
stock_prices = pd.read_csv('../InputData/stocks_prices_and_volumes.csv')
vix_indices = pd.read_csv('../InputData/vix_index.csv')
vixeem_indices = pd.read_csv('../InputData/vxeem_index.csv')
gold_prices = pd.read_csv('../InputData/intraday_gold.csv')
gold_prices_target = pd.read_csv('../InputData/target_gold.csv')
sentiment_data = pd.read_csv('./cleaned_sentiment_scores.csv')

In [104]:
stock_prices.head()

,Date,stock_0_food_beverages_and_tobacco_close_price,stock_10_non-banking_financial_services_close_price,stock_11_real_estate_close_price,stock_12_non-banking_financial_services_close_price,stock_13_it_&_media_close_price,stock_14_it_&_media_close_price,stock_1_energy_and_support_services_close_price,stock_2_basic_resources_close_price,stock_3_basic_resources_close_price,...,stock_14_it_&_media_volume,stock_1_energy_and_support_services_volume,stock_2_basic_resources_volume,stock_3_basic_resources_volume,stock_4_banks_volume,stock_5_banks_volume,stock_6_real_estate_volume,stock_7_real_estate_volume,stock_8_real_estate_volume,stock_9_non-banking_financial_services_volume
0,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02,15.24,2.23,8.15,8.20,NaN,8.00,3.64,21.05,9.80,...,48800.0,445065.0,13815.0,2264947.0,395.0,284645.0,241598.0,2390823.0,90156.0,22926.0
2,2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-05,14.84,2.10,7.58,8.17,NaN,7.62,3.32,19.74,8.88,...,264848.0,2740470.0,152334.0,1777066.0,88832.0,837444.0,3759516.0,16720687.0,1250162.0,1222827.0


In [253]:
predictions = []

# Convert the column to Date Object
sentiment_data['date'] = pd.to_datetime(sentiment_data['date'], errors='coerce')

gold_prices['Timestamp'] = pd.to_datetime(gold_prices['Timestamp'] , errors='coerce')

# Convert the column to Date Object
gold_prices['Date'] = gold_prices['Timestamp'].dt.date
sentiment_data['Date'] = sentiment_data['date'].dt.date

#Convert Column to Date Object
gold_daily = gold_prices.groupby('Date').last().reset_index()
sentiment_data = sentiment_data.groupby('Date').mean().reset_index()

sentiment_data = sentiment_data.sort_values(by='Date').reset_index(drop=True)
gold_daily = gold_daily.sort_values(by='Date').reset_index(drop=True)

    
# Keep only the 'Date' column and the specific stock column
sentiment_data_filtered = sentiment_data[['Date', 'sentiment_score']]
first_valid_index = sentiment_data_filtered['sentiment_score'].first_valid_index()
sentiment_data_filtered = sentiment_data_filtered.loc[first_valid_index:].reset_index(drop=True)


#Get Start and End Date relative to each dataset
start_date = max(sentiment_data_filtered['Date'].min(), gold_daily['Date'].min())
end_date = min(sentiment_data_filtered['Date'].max(), gold_daily['Date'].max())

#Filter the data based on the start and end date
sentiment_data_filtered = sentiment_data_filtered[(sentiment_data_filtered['Date'] >= start_date) & (sentiment_data_filtered['Date'] <= end_date)]
gold_daily = gold_daily[(gold_daily['Date'] >= start_date) & (gold_daily['Date'] <= end_date)]
# stock_prices_filtered[stock_prices_column] = stock_prices_filtered[stock_prices_column].interpolate(method='time')
# gold_daily['24K'] = gold_daily['24K'].interpolate(method='time')


#Create Log Columns
sentiment_data_filtered['Log_Sentiment_Score'] = np.log(sentiment_data_filtered['sentiment_score'] + 1)
gold_daily['Log_Gold_Price'] = np.log(gold_daily['24K'] + 1)

window_size = 30
num_std_devs = 2

gold_daily['Rolling_Mean_Gold'] = gold_daily['Log_Gold_Price'].rolling(window=window_size).mean()
gold_daily['Rolling_Std_Gold'] = gold_daily['Log_Gold_Price'].rolling(window=window_size).std()

gold_daily['Rolling_Upper_Threshold_Gold'] = gold_daily['Rolling_Mean_Gold'] + num_std_devs  * gold_daily['Rolling_Std_Gold']
gold_daily['Rolling_Lower_Threshold_Gold'] = gold_daily['Rolling_Mean_Gold'] - num_std_devs  * gold_daily['Rolling_Std_Gold']

sentiment_data_filtered['Rolling_Mean_Score'] = sentiment_data_filtered['Log_Sentiment_Score'].rolling(window=window_size).mean()
sentiment_data_filtered['Rolling_Std_Score'] = sentiment_data_filtered['Log_Sentiment_Score'].rolling(window=window_size).std()

sentiment_data_filtered['Rolling_Upper_Threshold_Score'] = sentiment_data_filtered['Rolling_Mean_Score'] + num_std_devs  * sentiment_data_filtered['Rolling_Std_Score']
sentiment_data_filtered['Rolling_Lower_Threshold_Score'] = sentiment_data_filtered['Rolling_Mean_Score'] - num_std_devs  * sentiment_data_filtered['Rolling_Std_Score']

outlier_mask_gold = (gold_daily['Log_Gold_Price'] < gold_daily['Rolling_Lower_Threshold_Gold']) | (gold_daily['Log_Gold_Price'] > gold_daily['Rolling_Upper_Threshold_Gold'])

outlier_mask_score = (sentiment_data_filtered['Log_Sentiment_Score'] < sentiment_data_filtered['Rolling_Lower_Threshold_Score']) | (sentiment_data_filtered['Log_Sentiment_Score'] > sentiment_data_filtered['Rolling_Upper_Threshold_Score'])

#Set the outliers to NaN
gold_daily.loc[outlier_mask_gold, 'Log_Gold_Price'] = np.nan
sentiment_data_filtered.loc[outlier_mask_score, 'Log_Stock_Price'] = np.nan


sentiment_data_filtered['Log_Sentiment_Score'] = sentiment_data_filtered['Log_Sentiment_Score'].shift(1)

#Merge the data
merged_data = pd.merge(sentiment_data_filtered, gold_daily, on='Date', how='outer')
#Remove duplicate dates
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
# merged_data = merged_data.drop_duplicates(subset='Date')

filtered_data = merged_data[['Date', 'Log_Sentiment_Score', 'Log_Gold_Price']]

filtered_data = filtered_data.set_index('Date')

filtered_data = filtered_data.reindex(date_range)
filtered_data['Log_Sentiment_Score'] = filtered_data['Log_Sentiment_Score'].interpolate(method='time')
filtered_data['Log_Gold_Price'] = filtered_data['Log_Gold_Price'].interpolate(method='time')

filtered_data = filtered_data.dropna(subset=['Log_Sentiment_Score', 'Log_Gold_Price'])

train_size = 0.8
split_index = int(len(filtered_data) * train_size)
train = filtered_data.iloc[:split_index].copy()
test = filtered_data.iloc[split_index:].copy()

df = pd.DataFrame()
df['y'] = train['Log_Gold_Price']
df['ds'] = train.index
df['Log_Sentiment_Score'] = train['Log_Sentiment_Score']
model = Prophet()
model.add_regressor('Log_Sentiment_Score')
model.fit(df)

test_size = len(test)
future_periods = test_size
prophet_test = test.reset_index().rename(columns={'index':'ds','Log_Gold_Price':'y'})
test_fcst = model.predict(prophet_test)
test_fcst['Predicted_Gold_Price'] = np.exp(test_fcst['yhat'])
predictions.append(test_fcst)

21:52:54 - cmdstanpy - INFO - Chain [1] start processing
21:52:54 - cmdstanpy - INFO - Chain [1] done processing


In [254]:
# Convert the column to Date Object
stock_prices['Date'] = pd.to_datetime(stock_prices['Date']).dt.date

gold_prices['Timestamp'] = pd.to_datetime(gold_prices['Timestamp'] , errors='coerce')

# Convert the column to Date Object
gold_prices['Date'] = gold_prices['Timestamp'].dt.date

#Convert Column to Date Object
gold_daily = gold_prices.groupby('Date').last().reset_index()

stock_prices = stock_prices.sort_values(by='Date').reset_index(drop=True)
gold_daily = gold_daily.sort_values(by='Date').reset_index(drop=True)
for stock_prices_column in stock_prices.columns:
    if stock_prices_column == 'Date':
        continue
    
    # Keep only the 'Date' column and the specific stock column
    stock_prices_filtered = stock_prices[['Date', stock_prices_column]]
    first_valid_index = stock_prices_filtered[stock_prices_column].first_valid_index()
    stock_prices_filtered = stock_prices_filtered.loc[first_valid_index:].reset_index(drop=True)
 

    
    #Get Start and End Date relative to each dataset
    start_date = max(stock_prices_filtered['Date'].min(), gold_daily['Date'].min())
    end_date = min(stock_prices_filtered['Date'].max(), gold_daily['Date'].max())
    
    #Filter the data based on the start and end date
    stock_prices_filtered = stock_prices_filtered[(stock_prices_filtered['Date'] >= start_date) & (stock_prices_filtered['Date'] <= end_date)]
    gold_daily = gold_daily[(gold_daily['Date'] >= start_date) & (gold_daily['Date'] <= end_date)]
    # stock_prices_filtered[stock_prices_column] = stock_prices_filtered[stock_prices_column].interpolate(method='time')
    # gold_daily['24K'] = gold_daily['24K'].interpolate(method='time')

    
    #Create Log Columns
    stock_prices_filtered['Log_Stock_Price'] = np.log(stock_prices_filtered[stock_prices_column] + 1)
    gold_daily['Log_Gold_Price'] = np.log(gold_daily['24K'] + 1)

    window_size = 30
    num_std_devs = 2
    
    gold_daily['Rolling_Mean_Gold'] = gold_daily['Log_Gold_Price'].rolling(window=window_size).mean()
    gold_daily['Rolling_Std_Gold'] = gold_daily['Log_Gold_Price'].rolling(window=window_size).std()
    
    gold_daily['Rolling_Upper_Threshold_Gold'] = gold_daily['Rolling_Mean_Gold'] + num_std_devs  * gold_daily['Rolling_Std_Gold']
    gold_daily['Rolling_Lower_Threshold_Gold'] = gold_daily['Rolling_Mean_Gold'] - num_std_devs  * gold_daily['Rolling_Std_Gold']
    
    stock_prices_filtered['Rolling_Mean_Stock'] = stock_prices_filtered['Log_Stock_Price'].rolling(window=window_size).mean()
    stock_prices_filtered['Rolling_Std_Stock'] = stock_prices_filtered['Log_Stock_Price'].rolling(window=window_size).std()
    
    stock_prices_filtered['Rolling_Upper_Threshold_Stock'] = stock_prices_filtered['Rolling_Mean_Stock'] + num_std_devs  * stock_prices_filtered['Rolling_Std_Stock']
    stock_prices_filtered['Rolling_Lower_Threshold_Stock'] = stock_prices_filtered['Rolling_Mean_Stock'] - num_std_devs  * stock_prices_filtered['Rolling_Std_Stock']
    
    outlier_mask_gold = (gold_daily['Log_Gold_Price'] < gold_daily['Rolling_Lower_Threshold_Gold']) | (gold_daily['Log_Gold_Price'] > gold_daily['Rolling_Upper_Threshold_Gold'])
    
    outlier_mask_stock = (stock_prices_filtered['Log_Stock_Price'] < stock_prices_filtered['Rolling_Lower_Threshold_Stock']) | (stock_prices_filtered['Log_Stock_Price'] > stock_prices_filtered['Rolling_Upper_Threshold_Stock'])
    
    #Set the outliers to NaN
    gold_daily.loc[outlier_mask_gold, 'Log_Gold_Price'] = np.nan
    stock_prices_filtered.loc[outlier_mask_stock, 'Log_Stock_Price'] = np.nan
    
    
    stock_prices_filtered['Log_Stock_Price'] = stock_prices_filtered['Log_Stock_Price'].shift(1)
    
    #Merge the data
    merged_data = pd.merge(stock_prices_filtered, gold_daily, on='Date', how='outer')
    #Remove duplicate dates
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    # merged_data = merged_data.drop_duplicates(subset='Date')
    
    filtered_data = merged_data[['Date', 'Log_Stock_Price', 'Log_Gold_Price']]
    
    filtered_data = filtered_data.set_index('Date')
    filtered_data = filtered_data.reindex(date_range)
    filtered_data['Log_Stock_Price'] = filtered_data['Log_Stock_Price'].interpolate(method='time')
    filtered_data['Log_Gold_Price'] = filtered_data['Log_Gold_Price'].interpolate(method='time')
    
    filtered_data = filtered_data.dropna(subset=['Log_Stock_Price', 'Log_Gold_Price'])
    
    train_size = 0.8
    split_index = int(len(filtered_data) * train_size)
    train = filtered_data.iloc[:split_index].copy()
    test = filtered_data.iloc[split_index:].copy()
    
    df = pd.DataFrame()
    df['y'] = train['Log_Gold_Price']
    df['ds'] = train.index
    df['Log_Stock_Price'] = train['Log_Stock_Price']
    model = Prophet()
    model.add_regressor('Log_Stock_Price')
    model.fit(df)
    
    test_size = len(test)
    future_periods = test_size
    prophet_test = test.reset_index().rename(columns={'index':'ds','Log_Gold_Price':'y'})
    test_fcst = model.predict(prophet_test)
    test_fcst['Predicted_Gold_Price'] = np.exp(test_fcst['yhat'])
    predictions.append(test_fcst)
    
predictions

21:52:57 - cmdstanpy - INFO - Chain [1] start processing
21:52:57 - cmdstanpy - INFO - Chain [1] done processing
21:52:58 - cmdstanpy - INFO - Chain [1] start processing
21:52:59 - cmdstanpy - INFO - Chain [1] done processing
21:52:59 - cmdstanpy - INFO - Chain [1] start processing
21:53:00 - cmdstanpy - INFO - Chain [1] done processing
21:53:00 - cmdstanpy - INFO - Chain [1] start processing
21:53:01 - cmdstanpy - INFO - Chain [1] done processing
21:53:01 - cmdstanpy - INFO - Chain [1] start processing
21:53:02 - cmdstanpy - INFO - Chain [1] done processing
21:53:02 - cmdstanpy - INFO - Chain [1] start processing
21:53:02 - cmdstanpy - INFO - Chain [1] done processing
21:53:02 - cmdstanpy - INFO - Chain [1] start processing
21:53:03 - cmdstanpy - INFO - Chain [1] done processing
21:53:03 - cmdstanpy - INFO - Chain [1] start processing
21:53:03 - cmdstanpy - INFO - Chain [1] done processing
21:53:04 - cmdstanpy - INFO - Chain [1] start processing
21:53:04 - cmdstanpy - INFO - Chain [1]

[            ds     trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
 0   2023-03-13  7.587401    7.585009    7.630681     7.587401     7.587401   
 1   2023-03-14  7.589538    7.582548    7.626312     7.589538     7.589538   
 2   2023-03-15  7.591675    7.585772    7.630597     7.591675     7.591675   
 3   2023-03-16  7.593813    7.585364    7.628704     7.593813     7.593813   
 4   2023-03-17  7.595950    7.585296    7.630386     7.595950     7.595950   
 ..         ...       ...         ...         ...          ...          ...   
 287 2023-12-25  8.200841    6.543258   10.017193     6.396683     9.862345   
 288 2023-12-26  8.202978    6.560717   10.033213     6.386570     9.881120   
 289 2023-12-27  8.205116    6.545317   10.041315     6.377391     9.894035   
 290 2023-12-28  8.207253    6.519019   10.044707     6.377401     9.899393   
 291 2023-12-29  8.209390    6.517499   10.059012     6.377946     9.912156   
 
      Log_Sentiment_Score  Log_Sentiment_Score_low

In [ ]:
gold_prices_target['Date'] = pd.to_datetime(gold_prices_target['date'])
gold_prices_target['Date'] = gold_prices_target['Date'].dt.date
gold_prices_target.drop(columns=['date'], inplace=True)
gold_prices_target.set_index('Date' , inplace=True)

In [ ]:
crude_oil_prices.head()

In [ ]:
crude_oil_prices['Date'] = pd.to_datetime(crude_oil_prices['Date'])
crude_oil_prices['Date'] = crude_oil_prices['Date'].dt.date
crude_oil_prices.set_index('Date' , inplace=True)

In [ ]:
features_df = pd.DataFrame(crude_oil_prices['Europe Brent Crude Oil (Dollars per Barrel)'].pct_change() * 100).shift(-1)
X = features_df[['Europe Brent Crude Oil (Dollars per Barrel)']]
X

# EDA

In [ ]:
gold_prices_target.describe()

In [ ]:
px.line(gold_prices_target)

In [ ]:
crude_oil_prices.describe()

In [ ]:
px.line(crude_oil_prices)

# Feature Selection

In [ ]:
features_df = pd.merge(gold_prices_target, (crude_oil_prices['Europe Brent Crude Oil (Dollars per Barrel)'].pct_change() * 100).shift(-1), on='Date', how='left')
features_df.fillna(0, inplace=True)
features_df

In [ ]:
px.line(features_df)

# Modelling

In [ ]:
X = features_df[['Europe Brent Crude Oil (Dollars per Barrel)']]  
y = features_df['pct_change']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

## Validation

In [ ]:
start_time = time.time()
y_pred = model.predict(X_test)
end_time = time.time()
inference_time = end_time - start_time

rmse = np.sqrt(mean_squared_error(y_test, y_pred))

mda = np.mean(np.sign(np.diff(y_test)) == np.sign(np.diff(y_pred)))

y_test_buckets = pd.qcut(y_test, 4, labels=False) 
y_pred_buckets = pd.qcut(y_pred, 4, labels=False)

f1 = f1_score(y_test_buckets, y_pred_buckets, average='weighted')

print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Directional Accuracy: {mda}')
print(f'Bucketized F1 Score: {f1}')
print(f'Inference Time (seconds): {inference_time}')


## Visualize the results

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(features_df.index, y, label='Actual Target Variable', color='blue')
plt.plot(X_test.index, y_pred, label='Predicted Target Variable', color='red')
plt.title('Linear Regression Model')
plt.xlabel('Date')
plt.ylabel('Target Variable')
plt.legend()
plt.show()

# Saving Model

In [ ]:
with open('../Pickles/linear_regression_model.pkl', 'wb') as file:
    pickle.dump(model, file)

# Loading Model

In [ ]:
with open('../Pickles/linear_regression_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

y_pred = loaded_model.predict(X_test)
output_df = pd.DataFrame({
    'date': X_test.index,
    'prediction': y_pred.flatten()
})
output_df